In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [3]:
full_df = pd.read_pickle('/data/newhouse/LRT/batch/master/training/tracks_weighted.pkl')

In [4]:
inputs_names = [ 
"z0",
"d0",
"numberOfSCTHits",
"Sihits",
"eta",
"numberDoF",
"geometric_var",
"numberOfSCTHoles",
]
label_names = ["is_good"]

df = full_df[inputs_names + label_names]
df

,z0,d0,numberOfSCTHits,Sihits,eta,numberDoF,geometric_var,numberOfSCTHoles,is_good
669124,148.534220,104.480734,7.0,9.0,-1.206826,6.0,1.947236,1.0,False
399311,-60.601026,-48.070017,9.0,10.0,1.381759,6.0,1.978903,0.0,False
313684,376.808424,99.458298,7.0,8.0,1.907424,4.0,0.036708,0.0,False
1583973,-81.997203,60.224389,8.0,9.0,1.302603,5.0,1.977285,0.0,True
2137964,-147.780404,21.521041,10.0,11.0,-2.370948,7.0,0.041632,0.0,True
...,...,...,...,...,...,...,...,...,...
2798810,-277.100815,24.420583,8.0,8.0,-2.205237,3.0,0.131661,0.0,True
425099,-327.316654,-19.400785,6.0,8.0,2.422333,5.0,2.905424,0.0,False
568234,66.349742,-19.168792,7.0,8.0,-0.830667,4.0,2.038449,1.0,False
2724619,-23.873419,-20.035454,8.0,8.0,0.082437,3.0,0.954928,0.0,True


In [5]:
train, test = train_test_split(df, test_size=0.2, random_state=1, shuffle=True)

In [ ]:
# Create model
def create_model():
    in_len = len(inputs_names)
    # Input layer
    inputs = keras.Input(shape=(len(inputs_names),))

    # Hidden layers
    dense = layers.Dense(in_len, activation="relu")
    x = dense(inputs)
    x = layers.Dense(in_len, activation="relu")(x)
    x = layers.Dense(in_len, activation="relu")(x)
    x = layers.Dense(in_len, activation="relu")(x)
    
    # Output layer
    outputs = layers.Dense(1,  activation="sigmoid")(x)    
    
    model = keras.Model(inputs=inputs, outputs=outputs, name="r3lrt_model")
    print(model.summary())
    
    return model

model = create_model()
keras.utils.plot_model(model, "r3lrt_model.png", show_shapes=True)

In [9]:
# Compile model
model.compile(loss='binary_crossentropy', 
              optimizer = keras.optimizers.Adamax(lr=0.0003),
              metrics=['accuracy', 'precision', 'recall'])

Model: "r3lrt_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 8)]               0         
_________________________________________________________________
dense_5 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_6 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_7 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_8 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 9         
Total params: 297
Trainable params: 297
Non-trainable params: 0
_________________________________________________________

In [10]:
history = model.fit(train[inputs_names], 
                    train[label_names], 
                    epochs=50, 
                    batch_size=500,
                    validation_data=(test[inputs_names], 
                                     test[label_names]),
#                     callbacks=[plot, early_stop, GetBest(monitor='val_acc', verbose=1, mode='max') ], 
#                     validation_data=(df[inputs_names][100:120], df[label_names][100:120], ),
#                     sample_weight = train_weights
                   )

Epoch 1/50


ValueError: in user code:

    /home/newhouse/miniconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /home/newhouse/miniconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/newhouse/miniconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/newhouse/miniconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/newhouse/miniconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:543 train_step  **
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    /home/newhouse/miniconda3/lib/python3.7/site-pack